# Demo SingleRun
This notebook shows a single run consisting of the following four phases:
* sail empty
* loading
* sail full
* unloading

In [1]:
import datetime, time
import simpy

import shapely.geometry
from simplekml import Kml, Style

import pandas as pd
import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot
import openclsim.plugins as plugin

from plot import vessel_planning

import matplotlib.pyplot as plt
%matplotlib inline

# setup environment
simulation_start  = datetime.datetime(2010,1,1)
my_env = simpy.Environment(initial_time=simulation_start.timestamp())
registry = {}
keep_resources = {}

import numpy as np

## Definition of Site

In [2]:
# The generic site class
Site = type(
    "Site",
    (
        core.Identifiable,  # Give it a name
        core.Log,  # Allow logging of all discrete events
        core.Locatable,  # Add coordinates to extract distance information and visualize
        core.HasContainer,  # Add information on the material available at the site
        core.HasResource,
    ),  # Add information on serving equipment
    {},
)  # The dictionary is empty because the site type is generic

# Information on the extraction site - the "from site" - the "win locatie"
location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)  # lon, lat

data_from_site = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Winlocatie",  # The name of the site
    "ID": "6dbbbdf4-4589-11e9-a501-b469212bff5d",  # For logging purposes
    "geometry": location_from_site,  # The coordinates of the project site
    "capacity": 40,  # The capacity of the site
    "level": 40,
}  # The actual volume of the site

# Information on the dumping site - the "to site" - the "dump locatie"
location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)  # lon, lat

data_to_site = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Dumplocatie",  # The name of the site
    "ID": "6dbbbdf5-4589-11e9-82b2-b469212bff5c",  # For logging purposes
    "geometry": location_to_site,  # The coordinates of the project site
    "capacity": 40,  # The capacity of the site
    "level": 0,
}  # The actual volume of the site (empty of course)

# The two objects used for the simulation
from_site = Site(**data_from_site)
to_site = Site(**data_to_site)

init
level: 40
completed init
init
level: 0
completed init


## Definition of Vessels

In [3]:
# The generic class for an object that can move and transport (a TSHD for example)
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,  # Give it a name
        core.Log,  # Allow logging of all discrete events
        core.ContainerDependentMovable,  # A moving container, so capacity and location
        core.Processor,  # Allow for loading and unloading
        core.HasResource,  # Add information on serving equipment
        core.HasCosts,  # Add information on costs
        core.LoadingFunction,  # Add a loading function
        core.UnloadingFunction,  # Add an unloading function
    ),
    {},
)

# For more realistic simulation you might want to have speed dependent on the volume carried by the vessel
def compute_v_provider(v_empty, v_full):
    return lambda x: 10

# TSHD variables
data_hopper = {
    "env": my_env,  # The simpy environment
    "name": "Hopper 01",  # Name
    "ID": "6dbbbdf6-4589-11e9-95a2-b469212bff5b",  # For logging purposes
    "geometry": location_from_site,  # It starts at the "from site"
    "loading_rate": 1,  # Loading rate
    "unloading_rate": 1,  # Unloading rate
    "capacity": 4,  # Capacity of the hopper - "Beunvolume"
    "compute_v": compute_v_provider(5, 4.5),  # Variable speed
    "weekrate": 7,
}

hopper = TransportProcessingResource(**data_hopper)

init
level: 0
completed init


## Definition of process

In [4]:
TestMoveActivity =  type(
    "TestMoveActivity",
    (
        plugin.HasTestPluginMoveActivity,
        plugin.HasWeatherPluginMoveActivity,
        model.MoveActivity,  # the order is critical!
    ),
    {},
)


TestShift =  type(
    "TestMoveActivity",
    (
        plugin.HasTestPluginMoveActivity,
        plugin.HasWeatherPluginMoveActivity,
        model.ShiftAmountActivity,  # the order is critical!
    ),
    {},
)

In [5]:
metocean_df = pd.read_csv("../demo/unit_test_weather.csv")
metocean_df = metocean_df.set_index(pd.to_datetime(metocean_df["Time"], dayfirst=True))
metocean_df = metocean_df.sort_index()

metocean_df["Hs [m]"] = 4 + \
                        1.5*np.sin(metocean_df['  Hour']/24 * 8 *np.pi) + \
                        1.5*np.sin(metocean_df['  Hour']/24 * 6 *np.pi)

metocean_df["DateTime"] = [datetime.datetime.strptime(dt, '%d-%m-%Y %H:%M') for dt in metocean_df["Time"]]

In [6]:
criteria_data_sail = {
    "event_name":"this",
    "condition":"Hs [m]",
    "maximum":3,
    "window_length":datetime.timedelta(hours=1)
}

sailing_crit = core.WorkabilityCriterion(**criteria_data_sail)

criteria_data_load = {
    "event_name":"this",
    "condition":"Hs [m]",
    "maximum":2,
    "window_length":datetime.timedelta(hours=1)
}

load_crit = core.WorkabilityCriterion(**criteria_data_load)

In [7]:
single_run = []

single_run.append(
    TestMoveActivity(**{
        "env": my_env,
        "name": "Soil movement",
        "registry": registry,
        "mover": hopper,
        "destination": from_site,
        "metocean_criteria":[sailing_crit],
        "metocean_df": metocean_df,
        "postpone_start": True,
    })
)

single_run.append(
    TestShift(**{
        "env": my_env,  # The simpy environment defined in the first cel
        "name": "Transfer MP",  # We are moving soil
        "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff52",  # For logging purposes
        "registry": registry,
        "processor": hopper,
        "origin": from_site,
        "destination": hopper,
        "amount": 4,
        "duration": 3600,
        "postpone_start": True,
        "metocean_criteria":[load_crit],
        "metocean_df": metocean_df,
    })
)

single_run.append(
    TestMoveActivity(**{
        "env": my_env,
        "name": "Soil movement",
        "registry": registry,
        "mover": hopper,
        "destination": to_site,
        "metocean_criteria":[sailing_crit],
        "metocean_df": metocean_df,
        "postpone_start": True,
    })
)

single_run.append(
    TestShift(**{
        "env": my_env,  # The simpy environment defined in the first cel
        "name": "Transfer TP",  # We are moving soil
        "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff54",  # For logging purposes
        "registry": registry,
        "processor": hopper,
        "origin": hopper,
        "destination": to_site,
        "amount": 4,
        "duration": 3600,
        "postpone_start": True,
        "metocean_criteria":[load_crit],
        "metocean_df": metocean_df,
    })
)

sequential_activity_data = {
    "env": my_env,
    "name": "Single run process",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff60",  # For logging purposes
    "registry": registry,
    "sub_processes": single_run,
    "postpone_start": True,
}
activity = model.SequentialActivity(**sequential_activity_data)

expr = [{"type":"container", "concept": to_site, "state":"full"}]
while_data = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "while",  # We are moving soil
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff5g",  # For logging purposes
    "registry": registry,
    "sub_process": activity,
    # "condition_event": [from_site.container.get_empty_event, to_site.container.get_full_event],
    #"condition_event": to_site.container.get_full_event,
    "condition_event": expr,
    "postpone_start": False,
}
while_activity = model.WhileActivity(**while_data)


check weather plugin
True
True
True
regrister weather plugin
check weather plugin
True
True
True
regrister weather plugin
check weather plugin
True
True
True
regrister weather plugin
check weather plugin
True
True
True
regrister weather plugin
while Activity keep_resources []
get_full_event : default
start get_available
start event instance None


In [8]:
my_env.run()

conditional 
start event instance None
keep_resources []
start event instance None
Mover_move before mover resource request
put_callback - id_ default
{}
put_callback - id_ default
{'default': {40: <Event() object at 0x1ad42b4fc88>}}
amount :40
put_callback - id_ default
{}
Mover_move after mover resource request
weatherPlugin start preprocess
[['2010-01-01T03:50:00.000000000' '2010-01-01T05:10:00.000000000']
 ['2010-01-01T14:50:00.000000000' '2010-01-01T15:10:00.000000000']
 ['2010-01-01T20:50:00.000000000' '2010-01-01T22:30:00.000000000']
 ['2010-01-02T03:50:00.000000000' '2010-01-02T05:10:00.000000000']
 ['2010-01-02T14:50:00.000000000' '2010-01-02T15:10:00.000000000']
 ['2010-01-02T20:50:00.000000000' '2010-01-02T22:30:00.000000000']
 ['2010-01-03T03:50:00.000000000' '2010-01-03T05:10:00.000000000']
 ['2010-01-03T14:50:00.000000000' '2010-01-03T15:10:00.000000000']
 ['2010-01-03T20:50:00.000000000' '2010-01-03T22:30:00.000000000']
 ['2010-01-04T03:50:00.000000000' '2010-01-04T05:10

start : {}
{'origin.6dbbbdf6-4589-11e9-95a2-b469212bff5b': 4, 'destination.6dbbbdf5-4589-11e9-82b2-b469212bff5c': 32}
destination request : {<simpy.resources.resource.Resource object at 0x000001AD41A9D548>: <Request() object at 0x1ad41a7d988>}
shift amount process keep_resources []
start get_available
processor request : {<simpy.resources.resource.Resource object at 0x000001AD41A9D548>: <Request() object at 0x1ad41a7d988>, <simpy.resources.resource.Resource object at 0x000001AD41AA3D88>: <Request() object at 0x1ad42b0c888>}
site request : {<simpy.resources.resource.Resource object at 0x000001AD41A9D548>: <Request() object at 0x1ad41a7d988>, <simpy.resources.resource.Resource object at 0x000001AD41AA3D88>: <Request() object at 0x1ad42b0c888>}
end requestIfAvailable : {<simpy.resources.resource.Resource object at 0x000001AD41A9D548>: <Request() object at 0x1ad41a7d988>, <simpy.resources.resource.Resource object at 0x000001AD41AA3D88>: <Request() object at 0x1ad42b0c888>}
after req resour

Mover_move after move
plugin_data mover: <__main__.TransportProcessingResource object at 0x000001AD41AA3A88>
plugin_data origin: <__main__.TransportProcessingResource object at 0x000001AD41AA3A88>
plugin_data destination: <__main__.Site object at 0x000001AD41A9D188>
plugin_data engine_oder: 1.0
plugin_data activity_log: <__main__.TestMoveActivity object at 0x000001AD42B1FB88>
plugin_data start_preprocessing: 1262470885.6491826
plugin_data start_activity: 1262486999.9999995
Activity end(Soil movement)
keep_resources []
SHift amount Activity keep_resources []
start event instance None
start : {}
{'origin.6dbbbdf4-4589-11e9-a501-b469212bff5d': 20, 'destination.6dbbbdf6-4589-11e9-95a2-b469212bff5b': 4}
destination request : {<simpy.resources.resource.Resource object at 0x000001AD41AA3D88>: <Request() object at 0x1ad41a86588>}
shift amount process keep_resources []
start get_available
processor request : {<simpy.resources.resource.Resource object at 0x000001AD41AA3D88>: <Request() object at

start : {}
{'origin.6dbbbdf4-4589-11e9-a501-b469212bff5d': 8, 'destination.6dbbbdf6-4589-11e9-95a2-b469212bff5b': 4}
destination request : {<simpy.resources.resource.Resource object at 0x000001AD41AA3D88>: <Request() object at 0x1ad4105b608>}
shift amount process keep_resources []
start get_available
processor request : {<simpy.resources.resource.Resource object at 0x000001AD41AA3D88>: <Request() object at 0x1ad4105b608>}
site request : {<simpy.resources.resource.Resource object at 0x000001AD41AA3D88>: <Request() object at 0x1ad4105b608>, <simpy.resources.resource.Resource object at 0x000001AD41A9D2C8>: <Request() object at 0x1ad42b0c288>}
end requestIfAvailable : {<simpy.resources.resource.Resource object at 0x000001AD41AA3D88>: <Request() object at 0x1ad4105b608>, <simpy.resources.resource.Resource object at 0x000001AD41A9D2C8>: <Request() object at 0x1ad42b0c288>}
after req resource if available : {<simpy.resources.resource.Resource object at 0x000001AD41AA3D88>: <Request() object a

In [9]:
log_df = pd.DataFrame(hopper.log)
data =log_df[['Message', 'ActivityState', 'Timestamp', 'Value', 'ActivityID']]
data = data.drop_duplicates()
data

,Message,ActivityState,Timestamp,Value,ActivityID
0,sailing empty,START,2010-01-01 03:50:00.000000,0,1a544728-abd3-11ea-adb6-2079183a5c9a
1,sailing empty,STOP,2010-01-01 03:50:00.000000,0,1a544728-abd3-11ea-adb6-2079183a5c9a
2,transfer default to Hopper 01,START,2010-01-01 03:50:00.000000,4,6dbbbdf7-4589-11e9-bf3b-b469212bff52
4,transfer default to Hopper 01,STOP,2010-01-01 04:50:00.000000,4,6dbbbdf7-4589-11e9-bf3b-b469212bff52
6,sailing filled,START,2010-01-01 04:50:00.000000,4,1a5a62fa-abd3-11ea-a255-2079183a5c9a
...,...,...,...,...,...
112,transfer default to Hopper 01,STOP,2010-01-04 22:05:42.824591,4,6dbbbdf7-4589-11e9-bf3b-b469212bff52
114,sailing filled,START,2010-01-04 22:05:42.824591,4,1a5a62fa-abd3-11ea-a255-2079183a5c9a
115,sailing filled,STOP,2010-01-04 22:21:25.649182,4,1a5a62fa-abd3-11ea-a255-2079183a5c9a
116,transfer default to Dumplocatie,START,2010-01-04 22:21:25.649182,4,6dbbbdf7-4589-11e9-bf3b-b469212bff54


In [11]:
import plotly.express as px

objects = [hopper, from_site, to_site]


activities = []
for obj in objects:
    activities.extend(set(obj.log["Message"]))

activities = list(set(activities))

fig = vessel_planning(objects, activities)

fig = px.line(metocean_df, x="DateTime", y="Hs [m]", title='Life expectancy in Canada')
fig.show()